# Heatmaps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from shapely import Point as ShapelyPoint
from shapely import Polygon as ShapelyPolygon
from matplotlib.patches import Polygon as PolygonPatch

from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
import time
from contextlib import contextmanager


@contextmanager
def timer():
    """Helper function to time code within a context block."""
    start_time = time.perf_counter()
    yield
    duration = time.perf_counter() - start_time
    print(f"Ran in {duration:0.1f} secs")

In [6]:
from ast import literal_eval


df_plays = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_play_metrics = pd.read_csv(f"{DIR}/data/processed/play_metrics.csv")
df_areas = pd.read_csv(f"{DIR}/data/processed/pocket_areas.csv")
df_areas["pocket"] = df_areas["pocket"].apply(literal_eval)

In [7]:
def get_play_pocket(df_play_metrics, df_areas, df_plays):
    """
    Choose one frame per play and area method and get
    the pocket object for that frame.
    """
    # Find the frame X seconds before the pocket ends.
    frames_per_second = 10
    df = pd.DataFrame(df_play_metrics)
    df = df[df["window_type"] == "before_end"]
    df["pocket_frame"] = (frames_per_second * df["time_start"]).astype(int)
    
    # Restrict the left side to one row per play, then
    # explode it with the right to get one row per play
    # and area type.
    left_cols = [
        "gameId",
        "playId",
        "pocket_frame",
        "frame_start",
        "frame_end",
    ]
    df = df[left_cols].drop_duplicates()

    # Extract vertices from pocket object, if any.
    def get_vertices(pocket):
        return pocket.get("metadata", {}).get("vertices")

    pocket_cols = [
        "gameId",
        "playId",
        "frameId",
        "method",
        "pocket",
    ]
    df_vertices = pd.DataFrame(df_areas[pocket_cols])
    vec_vertices = np.vectorize(get_vertices)
    df_vertices["vertices"] = vec_vertices(df_vertices["pocket"])
    df_vertices.drop(columns=["pocket"], inplace=True)
    
    # Drop plays without a polygon-shaped pocket.
    df_vertices = df_vertices[df_vertices["vertices"].notna()]

    # Inner join to keep only plays that have a window before
    # the pocket ends (left side) AND have a polygon-shaped
    # pocket (right side).
    df_pocket = df.merge(
        df_vertices,
        left_on=["gameId", "playId", "pocket_frame"],
        right_on=["gameId", "playId", "frameId"],
        how="inner",
    )
    df_pocket.drop(columns=["pocket_frame"], inplace=True)
    
    # Left join to plays to get metadata for these plays.
    df_out = df_pocket.merge(
        df_plays,
        on=["gameId", "playId"],
        how="left",
    )
    
    return df_out

In [8]:
show_cols = [
    "gameId",
    "playId",
    "frameId",
    "method",
    "vertices",
    "possessionTeam",
    "down",
]
df_pocket = get_play_pocket(df_play_metrics, df_areas, df_plays)
df_pocket[show_cols].head(10)

,gameId,playId,frameId,method,vertices,possessionTeam,down
0,2021090900,97,18,blocker_convex_hull,"[(-3.879999999999999, -3.1000000000000014), (-...",TB,3
1,2021090900,97,18,rushers_pocket_area,"[(-6.329999999999998, -2.1500000000000057), (-...",TB,3
2,2021090900,97,18,voronoi_pocket_area,"[(0.09839648090245734, -3.5495731919154774), (...",TB,3
3,2021090900,97,18,voronoi_rushers_only,"[(-0.7538752732534832, -3.145187813664214), (-...",TB,3
4,2021090900,97,18,adaptive_pocket_area,"[(-3.879999999999999, -3.1000000000000014), (-...",TB,3
5,2021090900,137,12,blocker_convex_hull,"[(-3.41, -1.8200000000000074), (-0.28999999999...",DAL,1
6,2021090900,137,12,rushers_pocket_area,"[(4.100000000000001, -0.38000000000000966), (-...",DAL,1
7,2021090900,137,12,voronoi_pocket_area,"[(-0.8616892212214013, -3.750377968366169), (-...",DAL,1
8,2021090900,137,12,voronoi_rushers_only,"[(-0.433970157080539, -2.7871829189807733), (3...",DAL,1
9,2021090900,137,12,adaptive_pocket_area,"[(-3.41, -1.8200000000000074), (-0.28999999999...",DAL,1
